# Training a Model

Now that I've got data in Hopsworks and the architecture for updating it, I can go ahead and start the process of constructing training data and then training a model.

In [1]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/14486
Connected. Call `.close()` to terminate connection gracefully.


In [2]:
# Load feature group.
zip_code = '60603'  # Chicago
country_code = 'US'
city = 'Chicago'

fg_name = f'aqi_{city}_{zip_code}'.lower()

aqi_online_fg = fs.get_feature_group(fg_name, version=1)

not_features = ['date', 'lat', 'lon']

ds_query = aqi_online_fg.select_except(not_features)

In [3]:
ds_query.show(5, online=True)

,co,no,no2,o3,so2,pm2_5,pm10,nh3,datetime,aqi,id
0,347.14,10.39,21.94,39.70,8.46,7.21,11.43,2.69,2020-11-28 12:00:00,1,36
1,440.60,16.99,26.39,26.82,10.97,18.80,24.32,4.37,2020-12-03 12:00:00,2,156
2,340.46,5.03,25.36,42.20,10.97,16.63,19.86,2.50,2020-12-03 16:00:00,2,160
3,273.71,0.02,14.91,41.84,5.42,11.97,13.51,0.63,2020-12-04 06:00:00,2,174
4,243.66,1.02,11.31,80.11,4.71,1.38,2.84,0.76,2020-12-05 12:00:00,2,204


Notice that the data appears to be out of order. This is ok.

I will now define some transformation functions to standardize all of our features. These transformations will be applied to the data when I create a feature view.

In [4]:
# Load the transformation function we want.
standard_scaler = fs.get_transformation_function(name="standard_scaler")

# Map features to transformation function
transformation_functions = {
    'co': standard_scaler, 
    'no': standard_scaler, 
    'no2': standard_scaler, 
    'o3': standard_scaler,
    'so2': standard_scaler, 
    'pm2_5': standard_scaler, 
    'pm10': standard_scaler, 
    'nh3': standard_scaler
}

Training data is created from feature views in Hopsworks. Feature views are logical views over sets of features. Normally they are created by joining together different feature groups. Since I only have one here though it's a little different.

In [5]:
fv_name = f'{fg_name}_fv'

try:
    feature_view = fs.get_feature_view(name=fv_name, version=1)
except: 
    feature_view = fs.create_feature_view(
    name=fv_name,
    version=1,
    description='feature view for creating training data',
    query=ds_query,
    labels=['aqi', 'id'],  # not using ID as a label, just for keeping track of data order
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/14486/fs/14406/fv/aqi_chicago_60603_fv/version/1


Now let's get the earliest and latest dates:

In [6]:
import datetime
import pandas as pd

end_date = pd.to_datetime(fs.sql(f"SELECT MAX(`datetime`) FROM `{fg_name}_1`", online=True).values[0][0])
start_date = pd.to_datetime(fs.sql(f"SELECT MIN(`datetime`) FROM `{fg_name}_1`", online=True).values[0][0])

print(start_date, end_date)

2020-11-27 00:00:00 2023-01-19 14:00:00


In [7]:
start_date_str = start_date.strftime('%Y-%m-%d %H:%M:%S')
end_date_str = end_date.strftime('%Y-%m-%d %H:%M:%S')

print(start_date_str, end_date_str)

2020-11-27 00:00:00 2023-01-19 14:00:00


In [8]:
# # Create training datasets based event time filter
train_d, train_d_job = feature_view.create_training_data(
        start_time = start_date_str,
        end_time = end_date_str,    
        description = f'aqi data for training {start_date_str} to {end_date_str}',
        data_format = "csv",
        coalesce = True,
        write_options = {'wait_for_job': False},
    )

Training dataset job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/14486/jobs/named/aqi_chicago_60603_fv_1_1_create_fv_td_21012023210052/executions


I'm going to use a portion of the dataset for normal training as well as evaluating the model with cross validation. Because of all this I'm not creating a separate training and testing set with Hopsworks's API.

Scikit-Learn has convenient functions for splitting time series data into validation sets, which I'll do later. For now, I just put all the data into one dataframe.

In [ ]:
train_x, train_y = feature_view.get_training_data(1)

In [ ]:
train_x.head()

In [ ]:
# check that we have the right time period for train and test
print(train_x['datetime'].min(), train_x['datetime'].max())

I'm still not sure why Hopsworks is returning the wrong subset of data. The dates are correct, but the times are getting converted to 12-hour time either when I create the training set or get the training set. You can see above that the beginning time is 12:00PM rather than 12:00AM (or 00:00 as I specified) and the ending time is 2:00AM rather than 2:00PM (I specified 14:00).

In [ ]:
train_x.dtypes

In [ ]:
# need to convert datetime from strings
train_x.datetime = pd.to_datetime(train_x.datetime)

In [ ]:
train_x.head()

In [ ]:
# data points are not in order
train_x = train_x.sort_values("datetime")
train_y = train_y.reindex(train_x.index)

In [ ]:
print(train_x['datetime'].min(), train_x['datetime'].max())

In [ ]:
# need to remove time zone information
train_x['datetime'] = train_x['datetime'].dt.tz_localize(None)

In [ ]:
print(train_x['datetime'].min(), train_x['datetime'].max())

In [ ]:
# use the datetime as index now
train_x = train_x.reset_index(drop=True)
train_x = train_x.set_index('datetime')

In [ ]:
train_y = train_y.reset_index(drop=True)
train_y = train_y.set_index(train_x.index)
train_y['aqi'] = train_y['aqi']-1  # xgboost requires zero indexed categories for classification

In [ ]:
train_x.tail()

In [ ]:
train_y.tail()

Now that the data is all cleaned up, I want to combine it all into a single dataframe. I'll call it `df` just so I don't get it confused between training, testing, validation, etc.

In [ ]:
# concat
df = pd.concat([train_x, train_y], axis=1)
df = df.drop(columns=['id'])

In [ ]:
df.head()

In [ ]:
df = train_y.drop(columns=['id'])

In [ ]:
df

In [ ]:
def create_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['hour'] = df.index.hour
    df['dayofweek'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    df['dayofmonth'] = df.index.day
    df['weekofyear'] = df.index.isocalendar().week
    return df

df = create_features(df)

In [ ]:
df

In [ ]:
df.columns

## Modeling with XGBoost

Now that I have data to train a model, I want to spend some time talking about model evaluation before I actually dive into training.

The target variable here is the air quality index (AQI), and in this case it ranges from (0 to 4) and is integral (the actual indices are 1 to 5, but I have to change it to 0 to 4 because XGBoost requires 0-indexed categories). Because of the integer constraint, I'm going to pose this machine learning task as a classification one. This will get rid of the need for rounding predictions to the nearest integer, but it also means I need to think a little bit more carefully about evaluating the model.

In classification tasks, model evaluation is extremely important. A common metric for evaluating classification models is accuracy. However, using that metric can cause modelers to fall into a trap of thinking a model is good, when in fact it's not doing any better than just guessing. For example, if I were to flip a fair coin 100 times, and then guess every time that the coin had landed on heads, I would have an accuracy of around 50%. Even though I just guessed every time, I still got an accuracy around 50%. In that case, a machine learning classifier would need to have an accuracy *greater than* 50% to be considered useful. That 50% accuracy is known as a baseline metric. This scenario gets more complicated when there are multiple classes and the classes are not balanced. 

The dataset at hand has a class imbalance, as shown below:

In [ ]:
df['aqi'].value_counts()/len(df['aqi'])

Say I just guess 0 (remember that's actually an AQI of 1) every time. Then I'll be correct 47.6% of the time. So I need to have a model that is correct more than that. What if I guess randomly, weighted by the share of the distribution each class has? That'll be:

In [ ]:
((df['aqi'].value_counts()/len(df['aqi']))**2).sum()

So basically, if I'm using accuracy as a metric to evaluate the model, I need to do better than 37.9% to be confident that the model is better than randomly guessing, and better than 47.6% to be better than just guessing the most common value. This is why accuracy is a misleading metric. A better metric to use would be the log-loss function. 

Recall that predictions with classifiers are made using probabilities, that is, the probability that a given record belongs to a particular class. For example in a binary classification problem, we predict the probablility that a record belongs to the positive class. The log-loss function measures how close a prediction probability is to the corresponding true value. The more a predicted probability diverges from the actual value, the higher the log-loss value is. In other words, a poorer prediction gets a higher log-loss score than a better one. 

The log-loss function $L\big(\hat{p}^{(i)}\big)$ is as follows for a two-class classification, where $\hat{p}^{(i)}$ is the probability that a given record belongs to the positive class and $y^{(i)}$ is the class (0 or 1):
$$L\big(\hat{p}^{(i)}\big)=-\Big(y^{(i)}\log\big(\hat{p}^{(i)}\big)+\big(1-y^{(i)}\big)\log\big(1-\hat{p}^{(i)}\big)\Big)$$

If I plot this for the positive class, $y^{(i)}=1$, we can see that poorer predictions are heavily penalized:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

n = 50
probs = np.linspace(0.01, 0.99, num=n)
y = np.full(n, 1, dtype=int)

def logloss(p, y):
    one = y * np.log(probs)
    two = (1-y)*np.log(1-probs)
    
    return -1*(one+two)

plt.plot(probs, logloss(probs, y), marker='o', color='r', label='Positive Class')
plt.legend()
plt.title('Log Loss of Prediction Probabilities \nfor a Positive (1) Class')
plt.show()

The log-loss metric is provided as a function in Scikit-Learn, which averages the log-loss for every record in the dataset.

Now the question is, what's the baseline log-loss for our dataset? Also, what is the log-loss when I just randomly predict the class, weighted by the share of each class in the distribution?

The baseline log-loss will be the log-loss when I predict a probability of a record belonging to the positive class 48% of the time, since the most prevalent class has a 48% share of the dataset:

In [ ]:
n = 100
probs = np.full(n, 0.48, dtype=float)
y = np.concatenate((np.full(48, 1, dtype=float), np.full(52, 0, dtype=float)))

logloss(probs, y).mean()

So I need a log-loss less than 0.69 for the model to be considered better than guessing 0 every time. Now what is the log loss when we just randomly predict the class, weighted by the share of each class in the distribution? That would just be the average of the log-losses for each class:

In [ ]:
from sklearn.metrics import log_loss

def calculate_log_loss(class_ratio, multi=10000):
    
    if sum(class_ratio)!=1.0:
        print("warning: Sum of ratios should be 1 for best results")
        class_ratio[-1]+=1-sum(class_ratio)  # add the residual to last class's ratio
    
    actuals=[]
    for i,val in enumerate(class_ratio):
        actuals=actuals+[i for x in range(int(val*multi))]
        

    preds=[]
    for i in range(multi):
        preds+=[class_ratio]

    return (log_loss(actuals, preds))



In [ ]:
calculate_log_loss([0.48, 0.38, 0.07, 0.06, 0.01])

So now I have an upper bound on the log-loss to determine if my model is worth it: 0.69. Now I can go ahead and start training the model.

## Training

Alright, so I need to do a little bit of preprocessing. I need to make some cross validation folds so that I can find the best model. After training, I'll write a function to compute some lag features so that I can make predictions into the future.

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
tss = TimeSeriesSplit(n_splits=3, test_size=24*90*1, gap=24)

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(15,15), sharex=True)

fold = 0
for train_idx, val_idx in tss.split(df):
    train_val = df.iloc[train_idx]
    test_val = df.iloc[val_idx]
    train_val['aqi'].plot(ax=axs[fold], 
                          label='Training Set', 
                          title=f'Cross Validation Fold {fold} Data Train/Test')
    test_val['aqi'].plot(ax=axs[fold], 
                          label='Test Set')
    axs[fold].axvline(test_val.index.min(), color='black', ls='--')
    fold += 1    

### Forecasting Horizon

I want to predict data 3 days into the future. Thus the forecasting horizon will be 3 days.

In [ ]:
def add_lags(df: pd.DataFrame) -> pd.DataFrame:
    target_map = df['aqi'].to_dict()
    df['aqi_lag3'] = (df.index - pd.Timedelta('3 days')).map(target_map)
    df['aqi_lag6'] = (df.index - pd.Timedelta('6 days')).map(target_map)
    df['aqi_lag9'] = (df.index - pd.Timedelta('9 days')).map(target_map)
    return df

In [ ]:
df = add_lags(df).copy()

In [ ]:
df.head()

In [ ]:
df['weekofyear'] = df['weekofyear'].astype('int64')

In [ ]:
df.dtypes

### Train with Cross Validation

Use cross validation to find some good parameters for the model without overfitting.

In [ ]:
import xgboost as xgb

In [ ]:
from sklearn.utils.class_weight import compute_sample_weight

In [ ]:
fold = 0
preds = []
scores = []

for train_idx, val_idx in tss.split(df):
    train = df.iloc[train_idx]
    test = df.iloc[val_idx]
    
    features = ['hour', 'dayofweek', 'quarter', 'month', 
                'year', 'dayofyear', 'dayofmonth', 'weekofyear', 'aqi_lag3', 'aqi_lag6', 'aqi_lag9']
    target = 'aqi'
    
    x_train = train[features]
    y_train = train[target]
    
    x_test = test[features]
    y_test = test[target]
    
    sample_weights = compute_sample_weight(class_weight='balanced', y=train[target])

    clf = xgb.XGBClassifier(n_estimators=1500, 
                            booster='gbtree',
                            early_stopping_rounds=50,
                            max_depth=3,
                            learning_rate=0.005)
    clf.fit(x_train, y_train,
            eval_set=[(x_train, y_train), (x_test, y_test)],
            verbose=100,
#             sample_weight=sample_weights
    )
    
    y_pred = clf.predict_proba(x_test)
    preds.append(y_pred)
    score = log_loss(y_test, y_pred)
    scores.append(score)

In [ ]:
print('CV Fold Scores: ', scores)
print(f'Average Score: {np.mean(scores):0.4f}')

### Retraining with Full Dataset

In [ ]:
features = ['hour', 'dayofweek', 'quarter', 'month', 
            'year', 'dayofyear', 'dayofmonth', 'weekofyear', 'aqi_lag3', 'aqi_lag6', 'aqi_lag9']
target = 'aqi'

x_all = df[features]
y_all = df[target]

sample_weights = compute_sample_weight(class_weight='balanced', 
                                       y=df[target])

clf = xgb.XGBClassifier(n_estimators=1500, 
                        booster='gbtree',
                        early_stopping_rounds=50,
                        max_depth=3,
                        learning_rate=0.005)

clf.fit(x_all, y_all, eval_set=[(x_all, y_all)], 
        verbose=100, sample_weight=sample_weights)

### Making Future DataFrame

In [ ]:
future_start = df.index.max()
future_end = future_start + pd.Timedelta('3 days') - pd.Timedelta('1 hours')

In [ ]:
future = pd.date_range(future_start, future_end, freq='1h')
future_df = pd.DataFrame(index=future)
future_df['isFuture'] = True
df['isFuture'] = False

df_and_future = pd.concat([df, future_df])

In [ ]:
df_and_future = add_lags(df_and_future)

In [ ]:
future_w_features = df_and_future.query('isFuture').copy()

In [ ]:
future_w_features = create_features(future_w_features)
future_w_features['weekofyear'] = future_w_features['weekofyear'].astype('int64')

In [ ]:
future_w_features.head()

In [ ]:
future_w_features['aqi_lag9'].value_counts()

In [ ]:
clf.predict(future_w_features[features])

In [ ]:
clf.feature_importances_

In [ ]:
importances = pd.DataFrame({'feature_importance': clf.feature_importances_}, index=features)

In [ ]:
importances